# Lab 1 discussion
Please set your OpenAI API key and AgentOps key in a local .env file for load_dotenv():
```
OPENAI_API_KEY=sk...
AGENTOPS_API_KEY=
```
or set it in the environment explicity using 
```
import os
os.environ["OPENAI_API_KEY"] = "sk..."
os.environ["AGENTOPS_API_KEY"] = ".."
```

In [9]:
import os
from dotenv import load_dotenv
import agentops

In [10]:
load_dotenv()
agentops.init(os.environ.get("AGENTOPS_API_KEY"), default_tags=["llm-agents-lab-01"])

🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=d3934a5e-f2b9-4206-8b66-ce102e8ed13e


In [3]:
from typing import Annotated, Literal
from typing import Dict
from typing import List

restaurant_datafile="restaurant-data.txt"

def normalize_restaurant_name(name):
    n = name.replace("-", " ")
    return n.strip().lower()

def get_substring_between(text, start_char, end_char):
    start_index = text.find(start_char)
    end_index = text.find(end_char, start_index + 1)

    # Check if both characters are found
    if start_index != -1 and end_index != -1:
        return text[start_index + 1: end_index ]
    else:
        return text  #just return the orignal text if [] are absent.

In [4]:
def load_restaurant_reviews() -> Dict[str, List[str]]:
    restaurant_map :Dict[str, List[str]] = {}
    with open(restaurant_datafile) as file:
        for line in file:
            # print(line)
            splits = line.split(".")
             # Use setdefault to ensure the key has a list, then append the value
            restaurant_map.setdefault(normalize_restaurant_name(splits[0]), []).append('.'.join(splits[1:]))
    # Iterate through the dictionary
    # for key, value in restaurant_map.items():
    #    print(f"Key: {key}, Value: {value}")
    return restaurant_map

def fetch_restaurant_data(restaurant_name:  Annotated[str, "restaurant name"]) -> Dict[str, List[str]]:
    global restaurant_name_str

    # This function takes in a restaurant name and returns the reviews for that restaurant. 
    # The output should be a dictionary with the key being the restaurant name and the value being a list of reviews for that restaurant.
    # The "data fetch agent" should have access to this function signature, and it should be able to suggest this as a function call. 
    # Example:
    # > fetch_restaurant_data("Applebee's")
    # {"Applebee's": ["The food at Applebee's was average, with nothing particularly standing out.", ...]}
    restaurant_name_str = restaurant_name
    name = normalize_restaurant_name(restaurant_name)
    reviews = load_restaurant_reviews()
    return {restaurant_name: reviews[name]}

In [5]:
fetch_restaurant_data("Starbucks")

{'Starbucks': [' Starbucks consistently serves good coffee and tasty pastries. The customer service is enjoyable, with baristas who are usually friendly and efficient.\n',
  ' The coffee was good and consistently prepared. The baristas provided amazing customer service, being both friendly and efficient.\n',
  ' Starbucks consistently delivers good coffee and pastries. The customer service was incredible, with friendly baristas who remembered my usual order.\n',
  " Starbucks consistently serves good coffee and tasty pastries. The customer service is awesome, with friendly baristas who remember regular customers' orders.\n",
  ' Both the food and service at Starbucks were great. The coffee was satisfying, and the barista was incredibly friendly and efficient.\n',
  ' Starbucks consistently serves good coffee and tasty snacks. The customer service is incredible, with friendly baristas who often remember regular customers.\n',
  ' Starbucks consistently delivers good coffee and pastries.

## Lab 1: Part 1 Two agent design to fetch reviews
Two Autogen ConversibleAgents to fetch reviews. We need a two agent design since one is a simple user agent used to
represent the user's queries. The second one is an LLM agent, using "gpt-4" as the LLM. Example shows "tool use" by registering
the tool description with the LLM agent, so that it can output a tool use call. The user agent receives that call and is able to execute the tool.

In [17]:
from autogen import ConversableAgent

ENTRYPOINT_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to fetch restaurants and answer questions based on provided restaurant
reviews. Use the tool calling to fetch reviews for a given restaurant requested by the user.
When the reviews are given as a list, copy them to the output for the next AI assistant to work on. Do not summarize.
"""
agentops.start_session()

restaurant_review_agent = ConversableAgent("Restaurant Review agent",
    system_message=ENTRYPOINT_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)


# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=0f8c5335-7bfa-487d-87e1-9687ce928fa2


In [18]:
# Tool use: https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use/
# Register the tool signature with the assistant agent.
restaurant_review_agent.register_for_llm(name="fetch_restaurant_reviews",
                                         description="Fetch reviews for a given restaurant name.")(fetch_restaurant_data)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="fetch_restaurant_reviews")(fetch_restaurant_data)

<function __main__.fetch_restaurant_data(restaurant_name: typing.Annotated[str, 'restaurant name']) -> Dict[str, List[str]]>

In [21]:
# Termination: use max_turns = 2
chat_result = user_proxy.initiate_chat(restaurant_review_agent, message="Fetch reviews for Starbucks.", max_turns=2)
agentops.end_session("Success")

User (to Restaurant Review agent):

Fetch reviews for Starbucks .

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_2wRRFFiI7VFohuR82v5CjNYu): fetch_restaurant_reviews *****
Arguments: 
{
"restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_2wRRFFiI7VFohuR82v5CjNYu) *****
{"Starbucks": [" Starbucks consistently serves good coffee and tasty pastries. The customer service is enjoyable, with baristas who are usually friendly and efficient.\n", " The coffee was good and consistently prepared. The baristas provided amazing customer service, being both friendly and efficient.\n",

🖇 AgentOps: Could not end session - multiple sessions detected. You must use session.end_session() instead of agentops.end_session() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


## Part 2: Compute a score for each review
We use the "Sequence" chat pattern from Autogen to send the fetched reviews to a scoring agent.
Example below shows how to send these to the food scoring agent. The prompt uses the food scoring logic to compute
a score.

In [22]:
# Sequence patterns: https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns
FOODSCORE_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to analyze and rate (from 1 to 5) the food service at a restaurant given the reviews.
Use the following table:
Score 1/5 has one of these adjectives: awful, horrible, or disgusting.
Score 2/5 has one of these adjectives: bad, unpleasant, or offensive.
Score 3/5 has one of these adjectives: average, uninspiring, or forgettable.
Score 4/5 has one of these adjectives: good, enjoyable, or satisfying.
Score 5/5 has one of these adjectives: awesome, incredible, or amazing.
Given a review, use the above table to output a rating as an integer. Only output the integer and nothing else.
"""

food_score_agent = ConversableAgent("Food Score agent",
    system_message=FOODSCORE_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [23]:
chat_result = user_proxy.initiate_chats(
[
        {
            "recipient": restaurant_review_agent,
            "message": "Whats the overall score for Taco Bell ?",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": food_score_agent,
            "message": "Given these reviews, generate a score for the food.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
])


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Restaurant Review agent):

Whats the overall score for Taco Bell ?

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_0M8snuu76juAvMdlGNFKBluU): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Taco Bell"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_0M8snuu76juAvMdlGNFKBluU) *****
{"Taco Bell": [" The food at Taco Bell was bad, with flavors that seemed artificial. The customer service w

## Review Scoring Agent
We modify the above prompt to return two scores, one for the food and one for customer service.

In [24]:
# Sequence patterns: https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns
REVIEWANALYSIS_AGENT_PROMPT = """
You are an helpful AI Assistant, with the ability to analyze and rate (from 1 to 5) the food service and customer service at a restaurant given the reviews.
Use the following table:
Score 1/5 has one of these adjectives: awful, horrible, or disgusting.
Score 2/5 has one of these adjectives: bad, unpleasant, or offensive.
Score 3/5 has one of these adjectives: average, uninspiring, or forgettable.
Score 4/5 has one of these adjectives: good, enjoyable, or satisfying.
Score 5/5 has one of these adjectives: awesome, incredible, or amazing.
Given a list of individual reviews, use the above table to output a rating as an integer along with the review. Output one integer each separately for food service and one for
customer service. Provide your output as a list of tuples (x,y, z) where x is the food service rating and y is the customer service rating for the review z.
"""
agentops.start_session()

review_analysis_agent = ConversableAgent("Review Analysis agent",
    system_message=REVIEWANALYSIS_AGENT_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=f6d7aa35-3088-4c34-b331-33da2f94433a


In [25]:
chat_result = user_proxy.initiate_chats(
[
        {
            "recipient": restaurant_review_agent,
            "message": "Whats the overall score for Starbucks ?",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": review_analysis_agent,
            "message": "Given these reviews, generate separate scores for the food and customer service.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
])
agentops.end_session("Success")


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Restaurant Review agent):

Whats the overall score for Starbucks ?

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_MS1CYsivOetITdyDB5L5Ce48): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Starbucks"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_MS1CYsivOetITdyDB5L5Ce48) *****
{"Starbucks": [" Starbucks consistently serves good coffee and tasty pastries. The customer service is enjo

🖇 AgentOps: Could not end session - multiple sessions detected. You must use session.end_session() instead of agentops.end_session() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


## Compute the final score

Routine to compute the final score and to process the summary message.

In [34]:
import re, math

def score_summary_method(sender: ConversableAgent, recipient: ConversableAgent, summary_args: dict):
    msg = recipient.last_message(sender)["content"]
    get_substring_between(msg, "[", "]")
    return "*************** Summary Method called:" + recipient.last_message(sender)["content"]

def extract_score(t):
    splits = t.replace("(", "").replace(")", "").split(",")
    f = int(splits[0])
    c = int(splits[1])
    return f, c

def calculate_overall_score(restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]) -> Dict[str, float]:
    # TODO
    # This function takes in a restaurant name, a list of food scores from 1-5, and a list of customer service scores from 1-5
    # The output should be a score between 0 and 10, which is computed as the following:
    # SUM(sqrt(food_scores[i]**2 * customer_service_scores[i]) * 1/(N * sqrt(125)) * 10
    # The above formula is a geometric mean of the scores, which penalizes food quality more than customer service. 
    # Example:
    # > calculate_overall_score("Applebee's", [1, 2, 3, 4, 5], [1, 2, 3, 4, 5])
    # {"Applebee's": 5.048}
    # NOTE: be sure to that the score includes AT LEAST 3  decimal places. The public tests will only read scores that have 
    # at least 3 decimal places.
    N = len(food_scores)
    if (N != len(customer_service_scores)):
        raise ValueError("Both food_scores and customer_service_scores must have equal length.")
    sum:float = 0.0
    for (f, c) in zip(food_scores, customer_service_scores):
        t = math.sqrt(f*f*c/125.0)
        sum = sum + t
        # print(f"Local score: {f}, {c}, {t}")
    #print(f"sum = {sum}, N = {N}")
    sum = sum / float(N)
    sum = 10.0 * sum
    print(f"Review for restaurant {restaurant_name} = {sum}")
    return "{\""+ restaurant_name + "\": "+ f"{sum:.3f}" + "}"

def compute_final_score(last_msg):
    global restaurant_name_str
    msg1 = get_substring_between(last_msg, "[", "]")
    res = re.findall(r'\(.*?\)', msg1)

    food_scores=[]
    service_scores = []
    for t in res:
        (f, c) = extract_score(t)
        food_scores.append(f)
        service_scores.append(c)
    return calculate_overall_score(restaurant_name_str, food_scores, service_scores)



In [35]:
user_query = "How good is the restaurant Chick-fil-A overall?" 
chat_result = user_proxy.initiate_chats([
        {
            "recipient": restaurant_review_agent,
            "message": user_query,
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": review_analysis_agent,
            "message": "Given these reviews, generate separate scores for the food and customer service.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },])

s = compute_final_score(chat_result[-1].summary)




********************************************************************************
Starting a new chat....

********************************************************************************
User (to Restaurant Review agent):

How good is the restaurant Chick-fil-A overall?

--------------------------------------------------------------------------------
Restaurant Review agent (to User):

***** Suggested tool call (call_5IU3rPKUDRMou32qoDn0BsFq): fetch_restaurant_reviews *****
Arguments: 
{
  "restaurant_name": "Chick-fil-A"
}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION fetch_restaurant_reviews...
User (to Restaurant Review agent):

User (to Restaurant Review agent):

***** Response from calling tool (call_5IU3rPKUDRMou32qoDn0BsFq) *****
{"Chick-fil-A": [" Chick-fil-A serves incredible chicken sandwiches that are always fresh and del